In [0]:
%run ./00_setup_and_config

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

In [0]:
raw_data_schema = StructType([
    StructField("InvoiceNo", StringType(), True),
    StructField("StockCode", StringType(), True),
    StructField("Description", StringType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("InvoiceDate", StringType(), True),
    StructField("UnitPrice", DoubleType(), True),
    StructField("CustomerID", StringType(), True),
    StructField("Country", StringType(), True)
])

In [0]:
try:
    df_raw = spark.read \
        .option("header", "True") \
        .schema(raw_data_schema) \
        .csv("/mnt/raw_data/OnlineRetail.csv")
except Exception as e:
    print("Error reading raw data: ", e)
    raise e


In [0]:
try:
    df_raw.write \
        .format("delta") \
        .mode("overwrite") \
        .option("mergeSchema", "true") \
        .save(raw_delta_path)
except Exception as e:
    print("Error writing raw data: ", e)